In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [2]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext
sqlContext=SQLContext(sc)

## ¿Cuál es la acción más realizada por usuarios que no están registrados?

Cargo el CSV desde el archivo parquet generado en el punto 1 y lo paso a RDD

In [6]:
df_logs = sqlContext.read.parquet('logs.parquet')
rdd_logs = df_logs.rdd

hago un map del RDD dejandome la estructura accion como clave y la ip como clave porque me piden de usuarios no registrados

In [7]:
rdd_logs_map = rdd_logs.map(lambda x: (x[7],x[4]))

Hago un filter por las ip que no sean None y por las acciones que tampoco sean None. Luego realizo un nuevo Map con la accion como clave y un 1 como valor para poder realizar el reduceByKey de todas las acciones.

In [10]:
rdd_actions = rdd_logs_map.filter(lambda x: x[1] != None or x[0]!=None).map(lambda x: (x[0],1)).reduceByKey(lambda x,y: x+y)

Hago el takeOrdered y traigo el que mas repeticiones tiene:

In [11]:
rdd_actions.takeOrdered(1, lambda x: -x[1])

[('create', 6065340)]

La accion mas realizada por usuarios no registrados es la de: CREATE